In [ ]:
import warnings; warnings.filterwarnings("ignore")

In [ ]:
from IPython.core.display import HTML; 
display(HTML("<style>.p-Widget.jp-OutputPrompt.jp-OutputArea-prompt:empty {padding: 0; border: 0;}</style>"));

In [ ]:
import os, sys
import json, pickle

import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm.auto import tqdm as tqdm

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
sys.path.append('../model_options')
from feature_extraction import *
from model_options import *

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Model Metadata Build

In [ ]:
model_options = get_model_options()

In [ ]:
def get_model_metadata(model_string, output = 'model+layer'):
    model_name = model_options[model_string]['model_name']
    train_type = model_options[model_string]['train_type']
    model = eval(model_options[model_string]['call'])
    
    model_metadata_dictlist = []
    layer_metadata_dictlist = []
    layer_metadata = get_feature_map_metadata(model)
    layer_count = len(layer_metadata.keys())
    layer_order = [model_layer for model_layer in layer_metadata]
    feature_counts = [layer_metadata[layer]['feature_count'] for layer in layer_metadata]
    parameter_counts = [layer_metadata[layer]['parameter_count'] for layer in layer_metadata]
    feature_map_shapes = [layer_metadata[layer]['feature_map_shape'] for layer in layer_metadata]
    total_feature_count = int(np.array(feature_counts).sum())
    total_parameter_count = int(np.array(parameter_counts).sum())
    model_metadata = {'total_feature_count': total_feature_count,
                      'total_parameter_count': total_parameter_count,
                      'layer_count': layer_count,
                      'layer_metadata': layer_metadata}

    model_metadata_dictlist.append({'model': model_name, 
                                    'train_type': train_type,
                                    'layer_count': layer_count,
                                    'total_feature_count': total_feature_count, 
                                    'total_parameter_count': total_parameter_count})

    model_metadata_frame = pd.DataFrame(model_metadata_dictlist)

    for layer_index, layer in enumerate(layer_metadata):
        layer_metadata_dictlist.append({'model': model_name,
                                        'train_type': train_type,
                                        'model_layer': layer, 
                                        'model_layer_index': layer_index + 1,
                                        'model_layer_depth': (layer_index + 1) / layer_count,
                                        'feature_count': layer_metadata[layer]['feature_count'],
                                        'parameter_count': layer_metadata[layer]['parameter_count']})

    layer_metadata_frame = pd.DataFrame(layer_metadata_dictlist)

    if output == 'json':
        return model_metadata
    if output == 'model_only':
        return model_metadata_frame
    if output == 'layer_only':
        return layer_metadata_frame
    if output == 'model+layer':
        return (model_metadata_frame, layer_metadata_frame)

In [ ]:
model_options = {**get_model_options(model_type='imagenet'), 
                 **get_model_options(train_type='taskonomy')}

model_metadata_dflist = []
layer_metadata_dflist = []

def process(model_option):
    incoming_metadata = get_model_metadata(model_option)
    model_metadata_dflist.append(incoming_metadata[0])
    layer_metadata_dflist.append(incoming_metadata[1])
    
problematic_model_options = []

def remark(model_option):
    problematic_model_options.append(model_option)
    
model_option_iterator = tqdm(model_options)
for model_option in model_option_iterator:
    #model_option_iterator.set_description(model_option)
    try: process(model_option)
    except: remark(model_option)

In [ ]:
problematic_model_options

In [ ]:
pd.concat(model_metadata_dflist).to_csv('model_metadata.csv', index = None)
pd.concat(layer_metadata_dflist).to_csv('layer_metadata.csv', index = None)

if 'model_metadata_json' in globals():
    with open('model_metadata.json', 'w') as file:
        json.dump(model_metadata_json, file)

In [ ]:
model_performance_data = pd.read_csv('model_scores.csv')#.set_index('model').to_dict('index')